<a href="https://colab.research.google.com/github/kkimtaejung/All_heuristic/blob/GAN/ROBO_Pytorch_NeuralNet_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GAN 첫걸음
### https://github.com/makeyourownneuralnetwork/gan

In [ ]:
import torch 
# 기본적인 숫자 이상의 정보를 저장 ex. 기울기
# 텐서(변수에 영향을 주고받는)에 대해 수식 등 추가적 정보 저장

# x가 3.5일 때, y의 그래프의 기울기
x = torch.tensor(3.5, requires_grad=True) # required_grad는 x에 대해 기울기 계산하도록 하는 기능
y = (x-1)*(x-2)*(x-3) #y = 1.8750

# 기울기에 대해 숫자로 계산해 텐서 x에 저장 : backward()
y.backward() #y' 에 x=3.5대입 -> 5.75
x.grad # x = 5.75

#연쇄법칙 - 기울기


##[1]
x=3.5에서 x에 대한 z' 미분값의 기울기 = dz/dx 

dz/dx = dz/dy*dy/dx 

dz/dx = 2*2x

In [ ]:
x = torch.tensor(3.5, requires_grad=True)
y = x*x
z = 2*y+3

#z의 기울기
z.backward()

x.grad #dz/dx = 14

##[2]
dz/dx = 2, dx/da = 2, dy/da = 10a

dz/dy = 3, dx/db = 3, dy/db = 9b^2

dz/da = (dz/dx*dx/da) + (dz/dy*dy/da) = 4 + 30a

In [ ]:
#[2]
a = torch.tensor(2.0, requires_grad=True)
b = torch.tensor(1.0, requires_grad=True)

x = 2*a + 3*b
y = 5*a*a + 3*b*b*b
z = 2*x + 3*y

#z기울기
z.backward()
#dz/da
a.grad

# MNIST 이미지 데이터셋

### 구글 드라이브 

In [ ]:
from google.colab import drive
drive.mount('./mount')

### 데이터 불러오기

In [ ]:
import pandas
df = pandas.read_csv('/content/mount/MyDrive/ROBOTICS/GAN/mnist_train.csv',header=None)
df.head()

In [ ]:
df.info()

### 이미지 확인을 위한 matplotlib

In [ ]:
import matplotlib.pyplot as plt

# dataframe에서 data 가져오기
row = 0
data = df.iloc[row]

# label가져오기
label = data[0]

# 784개 픽셀의 image data 보여주기
img = data[1:].values.reshape(28,28)
plt.title("label = "+str(label))
plt.imshow(img, interpolation='none', cmap='Blues')
plt.show()

### 간단한 신경망

입력 레이어 : 784개의 노드 필요

출력 레이어 : 10개의 노드 필요

노드 간 연결 : 완전연결구조

활성화 함수 : 로지스틱 함수

모델 구조 : 1 -> 2 -> 3 -> 4
1. 28*28(2차원)
2. 784(1차원 입력층)
3. 200(로지스틱 은닉층)
4. 10(로지스틱 출력층)


In [ ]:
import torch
import torch.nn as nn
class Classifier(nn.Module):
  def __init__(self):
    super().__init__() 
    self.model = nn.Sequential(
        nn.Linear(784,200),
        nn.LeakyReLU(0.02),
        nn.LayerNorm(200),
        nn.Linear(200,10),
        nn.Sigmoid()
    )
    self.loss_function = nn.MSELoss() 
    self.optimiser = torch.optim.Adam(self.parameters())
    self.counter = 0
    pass

  def forward(self,inputs):
    return self.model(inputs)

  def train(self, inputs, targets):
    outputs = self.forward(inputs)
    loss = self.loss_function(outputs, targets)

    self.optimiser.zero_grad()
    loss.backward()
    self.optimiser.step()

    self.counter += 1
    if(self.counter%10==0):
      self.progress.append(loss.item())
      pass
    
    if(self.counter%10000==0):
      print("counter = ",self.counter)
      pass

In [ ]:
import torch
import torch.nn as nn
# 어떤 신경망이든 torch.nn을 상속받아 클래스를 만들어야 함
# 상속받으면 계산 그래프, 훈련 시 가중치 조정 과정 자동화

class Classifier(nn.Module):
  # 부모 클래스 초기화
  # __init__(self)함수 : 객체가 처음 만들어질 때 호출되는 함수, 초기화 작업 = 생성자
  # super()는 생성자 호출 + 객체 생성까지 도와줌
  def __init__(self):
    super().__init__() 

    # 신경망 레이어 정의 (순방향)
    # 784노드 -> 200노드 -> 10 노드 구조
    # Sigmoid 로지스틱 활성화 함수
    # Linear는 선형 Ax+B 형태, A = weight, B = bias
    self.model = nn.Sequential(
        nn.Linear(784,200),
        nn.LeakyReLU(0.02),
        nn.LayerNorm(200),
        nn.Linear(200,10),
        nn.Sigmoid()
    )

    # 오차 정의 (역방향)
    # MSE 평균제곱오차 : (실제-예측)**2으로 계산 -> torch.nn.MSELoss()
    # SGD 확률적 경사 하강법 : 오차를 이용해 가중치 수정
    # learning rate : 0.01
    self.loss_function = nn.MSELoss() 
    self.optimiser = torch.optim.Adam(self.parameters())
    # 훈련 과정 시각화를 위한 코드
    self.counter = 0
    self.progress = []
    pass

  def forward(self,inputs):
      # 모델 실행 (순방향)
    return self.model(inputs)

  # train을 위한 메서드 생성
  # 메서드에는 입력, 출력이 필요
  def train(self, inputs, targets):
    # 신경망 출력 계산
    outputs = self.forward(inputs)
    # 손실 계산
    loss = self.loss_function(outputs, targets)

    # weight 업데이트 과정
    # 각 노드마다 오차 기울기 계산, 가중치 수정 과정 필요
    self.optimiser.zero_grad() # 기울기 초기화 -> 모두 0으로 초기화
    # 모두 0으로 초기화되지 않으면 loss.backward() 때마다 계산이 쌓임
    loss.backward() # 역전파 실행 -> 손실함수 계산
    self.optimiser.step() # 가중치 갱신

    # 카운터를 증가시키며 10회마다 오차 저장
    self.counter += 1
    if(self.counter%10==0):
      self.progress.append(loss.item()) # item은 텐서에서 값을 꺼내오는 함수
      pass
    # 10000번 훈련마다 훈련 횟수 출력
    if(self.counter%10000==0):
      print("counter = ",self.counter)
      pass
  
  #훈련과정 그림으로 표현
  def plot_progress(self):
    df = pandas.DataFrame(self.progress, columns=['loss'])
    df.plot(ylim=(0,1.0),figsize=(16,8),alpha=0.1,marker='.',grid=True,yticks=(0,0.25,0.5))
    pass
    # 1행은 차트로 나타내기 위해 손실값을 저장해둔 리스트를 데이터프레임으로 변환
    # 2행은 plot()함수로 여러 스타일과 디자인 지정

### torch로 데이터셋 불러오기

In [ ]:
from torch.utils.data import Dataset
# Dataset 클래승서 데이터셋 상속받기 위해 2개의 특수 메서드 구현 필요
# __len__() : 데이터셋 길이 반환
# __getitem__() : 데이터셋의 n번째 아이템 반환
class MnistDataset(Dataset):
  def __init__(self,csv_file):
    self.data_df = pandas.read_csv(csv_file, header=None)
    pass

  def __len__(self):
    return len(self.data_df)

  def __getitem__(self,index):
    #이미지 레이블
    label = self.data_df.iloc[index,0]
    target = torch.zeros((10))
    target[label] = 1.0
    #1의 경우 [0,1,0,0,0,0,0,0,0,0] -> 원핫 인코딩 됨

    #normalization
    image_values = torch.FloatTensor(self.data_df.iloc[index,1:].values)/255.0

    #레이블, 이미지데이터 텐서, 레이블 텐서 반환
    return label, image_values, target

  def plot_image(self, index):
    img = self.data_df.iloc[index,1:].values.reshape(28,28)
    plt.title("label = "+str(self.data_df.iloc[index,0]))
    plt.imshow(img, interpolation='none', cmap='Blues')
    pass
  pass

In [ ]:
mnist_dataset = MnistDataset('/content/mount/MyDrive/ROBOTICS/GAN/mnist_train.csv')

In [ ]:
mnist_dataset.plot_image(9)

### 신경망 훈련시키기 MNIST

In [ ]:
import matplotlib.pyplot as plt
import time

In [ ]:
#시간 얼마 걸렸는지 (훈련)
%%time

# 위에서 만든 신경망 call
C = Classifier()

# MNIST 데이터 훈련 진행
# mnist_dataset이 파이토치의 Dataset으로부터 상속받기 떄문에 for 반복문으로 훈련 데이터 처리 가능
epochs = 3

for i in range(epochs):
  print('traning epoch',i+1,"of",epochs)
  for label, image_data_tensor, target_tensor in mnist_dataset:
    C.train(image_data_tensor, target_tensor)
    pass
  pass

### 오차 출력

In [ ]:
C.plot_progress()

### 신경망 테스트

In [ ]:
mnist_test_dataset = MnistDataset('/content/mount/MyDrive/ROBOTICS/GAN/mnist_test.csv')

# 인덱스가 19인 이미지
record = 19

# 이미지, 정답 출력
mnist_test_dataset.plot_image(record)

In [ ]:
# 19 레코드의 이미지 픽섹 값을 추출해 image_data에 저장
image_data = mnist_test_dataset[record][1]
# forward 함수를 통해 전달
output = C.forward(image_data)
# output을 단순한 넘파이 행렬로 변환 후 다시 데이터프레임으로 래핑하여 막대도표로 표현
pandas.DataFrame(output.detach().numpy()).plot(kind='bar',legend=False,ylim=(0,1))

### 신경망 정확도

In [ ]:
score = 0
items = 0

for label, image_data_tensor, target_tensor in mnist_test_dataset:
  answer = C.forward(image_data_tensor).detach().numpy()
  if(answer.argmax() == label):
    score += 1
    pass
  items += 1
  pass
print(score, items, "정학도 : ",score/items*100)

### 회귀 vs 분류

In [ ]:
# 손실함수를 회귀 문제의 경우 : MSELoss() / 분류 문제의 경우 : BCELoss()
# 활성화 함수의 경우 기울기 소멸의 문제 : Sigmoid -> ReLU (0보다 작을 때 기울기 소실 문제) -> LeakyReLU(a)

### 옵티마이저

In [ ]:
# SGD(확률적 경사 하강법) : 국소 최적해에 빠질 확률이 높다, 모든 학습 파라미터에 단일한 학습률을 적용해버린다
#   self.optimiser = torch.optim.SGD(self.parameters)
# Adam : 국소 최적해로 빠질 가능성 줄임, 모든 학습 파라미터에 대해 각각 다른 학습률을 적용하여 상황에 따라 수정
#   self.optimiser = torch.optim.Adam(self.parameters)

### 정규화

파라미터의 범위를 줄여, 0-1사이의 값으로 만들어줌

신경망 레이어 설정 -> nn.LayerNorm(200)

# CUDA 기초

### numpy vs torch 행렬 계산 속도

넘파이가 순수 행렬곱 계산보다 훨씬 빠르다

### 엔비디아 CUDA

#### ::CPU:: (중앙처리장치)

- 컴퓨터의 대부분의 연산 처리

- 범용적으로 여러 가지 일 처리 가능

#### ::GPU:: (그래픽 처리 장치)

- 특정한 작업만 처리 가능

- 단순 계산 같은 병렬 행력곱 처리에 특화

#### ::CPU vs GPU::

- 연산 양이 어마어마 하다면,

- CPU는 2-16 코어 할당하여 하나하나 계산

- GPU는 천개 이상의 연산 코어 존재, 많은 연산이 쪼개져 병렬로 계산

#### 엔비디아

- 강력한 하드웨어 가속 기능을 갖춘, 머신러닝 연구에 표준적인 소프트웨어 툴 제공
- 소프트웨어 프레임워크 : CUDA
- CUDA는 GPU에서만 동작함

### CUDA GPU로 계산

In [ ]:
import torch
x = torch.cuda.FloatTensor([3.5])
x.type()

In [ ]:
x.device

In [ ]:
y = x*x
y

In [ ]:
import numpy
size = 600
a = numpy.random.rand(size,size)
b = numpy.random.rand(size,size)
aa = torch.cuda.FloatTensor(a)
bb = torch.cuda.FloatTensor(b)
cc = torch.matmul(aa,bb)
cc

### CUDA 가 사용 가능한지 확인

In [ ]:
if torch.cuda.is_available():
  torch.set_default_tensor_type(torch.cuda.FloatTensor)
  print("using cuda:", torch.cuda.get_device_name(0))
  pass
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device